In [1]:
import sqlite3
conn = sqlite3.connect("playlist.db")
cur = conn.cursor()

In [2]:
cur.execute('''
  CREATE TABLE ARTIST(
    PK INTEGER PRIMARY KEY,
    NAME TEXT NOT NULL DEFAULT '가수'
  );
''')

In [3]:
cur.executescript('''
  CREATE TABLE ALBUM(
    PK INTEGER PRIMARY KEY,
    NAME TEXT,
    FK INTEGER NOT NULL
  );
  CREATE TABLE GENRE(
    PK INTEGER PRIMARY KEY,
    NAME TEXT
  );
  CREATE TABLE TRACK(
    PK INTEGER PRIMARY KEY,
    NAME TEXT,
    LENGTH INTEGER DEFAULT 0,
    RATING INTEGER DEFAULT 0,
    COUNT INTEGER DEFAULT 0,
    AFK INTEGER NOT NULL,
    GFK INTEGER NOT NULL
  );
''')

# STAR RATING : 현업에서 사용자의 반응을 알기 위해 많이 사용한다. 보통 별 5개를 사용 => 통계기법

In [17]:
DB Broswer for SQLite

SyntaxError: invalid syntax (2602426753.py, line 1)

In [4]:
cur.executemany('''
    INSERT INTO ARTIST VALUES(NULL,?);
    ''',[['수지'],['박효신'],['아이유']])

In [5]:
conn.commit()

In [6]:
cur.executemany('''
    INSERT INTO GENRE VALUES(NULL,?);
''',[['발라드'],['R&B'],['락']])

In [7]:
conn.commit()

In [8]:
# artist에서 pk 찾아서 insert 
def addAlbum(artist, album):
    cur.execute('''SELECT PK FROM ARTIST WHERE NAME LIKE ?''', ['%'+artist+'%'])
    PK = cur.fetchone()

    if PK:
        FK = PK[0]
        cur.execute('INSERT INTO ALBUM VALUES(NULL, ?, ?)',(album, FK))
    
    return PK, artist, album, cur.lastrowid

In [35]:
addAlbum('박효신','박효신앨범')
addAlbum('아이유','아이유앨범')
conn.commit()

In [24]:
# INSERT track AFk=PK1, GFK=PK2
def addTrack(album, genre, track, length=0, rating=0,  count=0):
    cur.execute('SELECT PK FROM ALBUM WHERE NAME LIKE ?',['%'+album+'%'])
    PK1 = cur.fetchone()

    cur.execute('SELECT PK FROM GENRE WHERE NAME LIKE ?',['%'+genre+'%'])
    PK2 = cur.fetchone()

    if PK1 != None and PK2 != None:
        AFK = PK1[0]
        GFK = PK2[0]
        cur.execute('''
            INSERT INTO TRACK
            VALUES(NULL, :name, :length, :rating, :count, :afk, :gfk)
        ''', {'name':track, 'length':length, 'rating':rating, 'count':count,'afk':AFK, 'gfk':GFK })
        conn.commit()
        
    return PK1, PK2

In [37]:
addTrack('아이유','발라드','아이유노래1')
addTrack('아이유','발라드','아이유노래2')
addTrack('아이유','발라드','아이유노래3')

((3,), (1,))

In [36]:
addTrack('박효신','R&B','박효신노래1')
addTrack('박효신','발라드','박효신노래2')
addTrack('박효신','락','박효신노래3')

((2,), (3,))

In [12]:
cur.execute('SELECT * FROM ARTIST')
rst = cur.fetchall()
print(rst)

[(1, '수지'), (2, '박효신'), (3, '아이유')]


In [13]:
cur.execute('SELECT * FROM GENRE')
rst = cur.fetchall()
print(rst)

[(1, '발라드'), (2, 'R&B'), (3, '락')]


In [14]:
cur.execute('SELECT * FROM ALBUM')
rst = cur.fetchall()
print(rst)

[(1, '수지앨범', 1), (2, '아이유앨범', 3), (3, '박효진앨범', 2)]


In [15]:
cur.execute('SELECT * FROM TRACK')
rst = cur.fetchall()
print(rst)

[(1, '아이유노래1', 0, 0, 0, 2, 1), (2, '아이유노래2', 0, 0, 0, 2, 1), (3, '아이유노래3', 0, 0, 0, 2, 1)]


In [19]:
conn.commit()

In [34]:
cur.execute('DELETE FROM ALBUM WHERE NAME LIKE "%아이유%"')
conn.commit()

In [28]:
cur.execute('DELETE FROM TRACK WHERE NAME LIKE "%아이유%"')

In [29]:
conn.commit()

In [41]:
# 아티스트 - 앨범
cur.execute('SELECT DISTINCT * FROM ARTIST')
cur.fetchall()

cur.execute('SELECT * FROM ALBUM')
cur.fetchall()

cur.execute('''
    SELECT ARTIST.NAME, ALBUM.NAME
    FROM ARTIST
    LEFT JOIN ALBUM
    ON ALBUM.FK = ARTIST.PK
''')
cur.fetchall()


[('수지', '수지앨범'), ('박효신', '박효신앨범'), ('아이유', '아이유앨범')]

In [39]:
# 가수당 몇개의 앨범이 있는가? 
cur.execute('''
    SELECT ARTIST.NAME, COUNT(ALBUM.NAME)
    FROM ARTIST
    INNER JOIN ALBUM
    ON ALBUM.FK = ARTIST.PK
    GROUP BY ALBUM.NAME
    ORDER BY ARTIST.NAME ASC
''')
cur.fetchall()

[('박효신', 1), ('수지', 1), ('아이유', 1)]

In [47]:
# 앨범별 노래가 몇개 있는가? 
cur.execute('''
    SELECT ALBUM.NAME, COUNT(TRACK.NAME)
    FROM TRACK
    INNER JOIN ALBUM
    ON ALBUM.PK = TRACK.AFK
    GROUP BY ALBUM.NAME
''')
cur.fetchall()

[('박효신앨범', 3), ('아이유앨범', 3)]

In [46]:
# 장르별 몇개의 노래가 있는가? 
# GENRE = TRACK
cur.execute('''
    SELECT GENRE.NAME, COUNT(TRACK.NAME)
    FROM TRACK
    LEFT JOIN GENRE
    ON GENRE.PK = TRACK.AFK
    GROUP BY GENRE.NAME
''')
cur.fetchall()

[('R&B', 3), ('락', 3)]

In [48]:
# 기준이 되는 테이블을 아는 방법은?

In [52]:
# ARTIST - ALBUM - TRACK
cur.execute('''
    SELECT A.NAME, B.NAME, C.NAME
    FROM ARTIST AS A
    LEFT JOIN ALBUM AS B
    ON B.FK = A.PK
    LEFT JOIN TRACK AS C
    ON C.AFK = B.PK
''')
cur.fetchall()

[('수지', '수지앨범', None),
 ('박효신', '박효신앨범', '박효신노래1'),
 ('박효신', '박효신앨범', '박효신노래2'),
 ('박효신', '박효신앨범', '박효신노래3'),
 ('아이유', '아이유앨범', '아이유노래1'),
 ('아이유', '아이유앨범', '아이유노래2'),
 ('아이유', '아이유앨범', '아이유노래3')]

In [54]:
# ARTIST - ALBUM - TRACK
# 누구와 합칠지 먼저 구하고, 그 다음 함치는 것
cur.execute('''
    SELECT A.NAME, B.NAME
    FROM ARTIST AS A
    LEFT JOIN (
        SELECT ALBUM.FK, ALBUM.NAME AS NAME, TRACK.NAME AS TNAME
        FROM ALBUM
        LEFT JOIN TRACK
        ON TRACK.AFK = ALBUM.PK
    ) AS B
    ON B.FK = A.PK
''')
cur.fetchall()

[('수지', '수지앨범'),
 ('박효신', '박효신앨범'),
 ('박효신', '박효신앨범'),
 ('박효신', '박효신앨범'),
 ('아이유', '아이유앨범'),
 ('아이유', '아이유앨범'),
 ('아이유', '아이유앨범')]

In [56]:
# 
cur.execute('''
    SELECT A.NAME, B.NAME, COUNT(C.NAME)
    FROM ARTIST AS A, ALBUM AS B, TRACK AS C
    WHERE A.PK=B.FK AND B.PK=C.AFK
    GROUP BY B.NAME
''')
cur.fetchall()

[('박효신', '박효신앨범', 3), ('아이유', '아이유앨범', 3)]

In [59]:
# 4개 섞기
# ARTIST - ALBUM - GENRE - TRACK
# where 절에 조건을 붙여 불러오는 것은 비효율적
# row 데이터는 최대한 살려야 한다.
cur.execute('''
    SELECT A.NAME, B.NAME, C.NAME, D.NAME
    FROM ARTIST AS A, ALBUM AS B, GENRE AS C, TRACK AS D
    WHERE A.PK=B.FK AND B.PK=D.AFK AND C.PK=D.GFK
''')
cur.fetchall()

[('박효신', '박효신앨범', 'R&B', '박효신노래1'),
 ('박효신', '박효신앨범', '발라드', '박효신노래2'),
 ('박효신', '박효신앨범', '락', '박효신노래3'),
 ('아이유', '아이유앨범', '발라드', '아이유노래1'),
 ('아이유', '아이유앨범', '발라드', '아이유노래2'),
 ('아이유', '아이유앨범', '발라드', '아이유노래3')]

In [62]:
cur.execute('''
    SELECT A.NAME, B.NAME, C.NAME, COUNT(C.NAME), D.NAME,COUNT(D.NAME)
    FROM ARTIST AS A, ALBUM AS B, GENRE AS C, TRACK AS D
    WHERE A.PK=B.FK AND B.PK=D.AFK AND C.PK=D.GFK
    GROUP BY A.PK, B.PK, C.PK
''')
# GROUP BY A.PK, B.PK, C.PK 하나 씩 추가하면서 연습해보기
# 피봇 데이블을 만들수 있다. 새로운 기준으로 카운트를 셀수 있다. 이걸 pandas를 쓴다.
# 그런데, 이것도 알아야한다. 파이선의 자원 뿐아니라 서버의 자원을 충분히 활용할 수 있어야 한다.
cur.fetchall()

[('박효신', '박효신앨범', '발라드', 1, '박효신노래2', 1),
 ('박효신', '박효신앨범', 'R&B', 1, '박효신노래1', 1),
 ('박효신', '박효신앨범', '락', 1, '박효신노래3', 1),
 ('아이유', '아이유앨범', '발라드', 3, '아이유노래1', 3)]

In [63]:
cur.execute('''
    CREATE VIEW ALBUM_TRACK AS 
    SELECT ALBUM.FK, ALBUM.NAME AS NAME, TRACK.NAME AS TNAME
        FROM ALBUM
        LEFT JOIN TRACK
        ON TRACK.AFK = ALBUM.PK 
''')

In [64]:
cur.execute('SELECT * FROM ALBUM_TRACK')
cur.fetchall()
# 뷰 테이블을 사용하면 좋다.
# 1) 원본 데이터를 숨길 수 있다.

[(1, '수지앨범', None),
 (2, '박효신앨범', '박효신노래1'),
 (2, '박효신앨범', '박효신노래2'),
 (2, '박효신앨범', '박효신노래3'),
 (3, '아이유앨범', '아이유노래1'),
 (3, '아이유앨범', '아이유노래2'),
 (3, '아이유앨범', '아이유노래3')]

In [65]:
# 중첩과 조인을 이어서 사용하는 것 중에서 무엇이 더 빠른가? 조인을 이어쓰는게 빠르다.
# 조인은 수정이 어렵고 헷갈린다. 자신이 없다면 뷰테이블을 만드는 것이 좋다.

In [66]:
# 기준이 되는 칼럼이란 미싱 벨류가 사는 쪽
# SELECT * FORM A LEFT JOIN B -> A의 모든 값이 산다, 원본이 된다.
# SELECT * FORM A RIGHT JOIN B -> B의 모든 값이 산다, 원본이 된다.

In [67]:
conn.close()

In [74]:
# SNS - 게시글(해시태그) 등록/수정/
conn = sqlite3.connect('sns.db')
cur = conn.cursor()

In [75]:
cur.executescript('''
    CREATE TABLE POST(
        PK INTEGER PRIMARY KEY
    );
    CREATE TABLE HASHTAG(
        PK  INTEGER PRIMARY KEY,
        TAG TEXT NOT NULL,
        COUNT INTEGER DEFAULT 0      
    );
    CREATE TABLE POSTTAG(
        PFK INTEGER NOT NULL,
        TFK INTEGER NOT NULL
    )
''')
conn.commit()

In [76]:
cur.execute('''
    ALTER TABLE POST
    ADD COLUMN CONTENT TEXT
''')

In [73]:
conn.close()

In [77]:
conn = sqlite3.connect('playlist.db')
cur = conn.cursor()

In [78]:
cur.execute('''
    ALTER TABLE ARTIST
    ADD COLUMN temp TEXT NOT NULL DEFAULT '';
''')

In [79]:
conn.close()

In [80]:
conn = sqlite3.connect('sns.db')
cur = conn.cursor()

In [88]:
# POST 등록하기
def newPosting(content, *tags):

    # 1
    cur.execute('INSERT INTO POST VALUES (NULL,?)',(content,))
    conn.commit()

    cur.execute('SELECT PK FROM POST ORDER BY PK DESC LIMIT 0, 1')
    fk = cur.fetchone()[0]

    # 2
    fkList = list()
    for tag in tags:
        # 기본 태그풀에 있는지 단순 검사
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?',(tag,))
        PK = cur.fetchone()
        if PK is None:
            cur.execute('INSERT INTO HASHTAG VALUES (NULL, ?, NULL)',(tag,))
            conn.commit()
        
        # 무조건 태그풀에 태그가 있음
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?',(tag,))
        PK2 = cur.fetchone()
        fkList.append(PK2)
        cur.execute('UPDATE HASHTAG SET COUNT = COUNT +1 WHERE PK=?',(PK2,))
        conn.commit()

    # 3
    for FK in fkList:
        cur.executemany('INSERT INTO POSTTAG VALUES(?,?)',(fk,FK)) 
    
    conn.commit()
    
    return PK, fkList

newPosting("내용", '태크1', '태그2')

ProgrammingError: Error binding parameter 1: type 'tuple' is not supported

In [82]:
# 등롥 - 1 (내용 추가)
#cur.execute('INSERT INTO POST VALUES (NULL,?)',(content,))
#conn.commit()
#print(cur.lastrowid) # pk 값 으로 사용금지

#cur.execute('SELECT PK FROM POST WHERE CONTENT=?',(content,))
cur.execute('SELECT PK FROM POST ORDER BY PK DESC LIMIT 0, 1')
conn.commit()
PK = cur.fetchall()

In [85]:
# 등록 - 2 (해시태그 추가)
# fkList = list()
# for tag in tags:
#     cur.execute('SELECT PK FROM HASHTAG TAG=?',(tag,))
#     PK = cur.fetchone()
#     if PK:
#         PK = PK[0]
#         fkList.appned(PK)
#         cur.execute('UPDATE HASHTAG SET COUNT = COUNT + 1 WHERE PK=?',(PK,))
#         conn.commit()
#     else:
#         cur.execute('INSERT INTO HASHTAG VALUES (NULL, ?, 1)',(tag,))
#         conn.commit()

fkList = list()
for tag in tags:
    # 기본 태그풀에 있는지 단순 검사
    cur.execute('SELECT PK FROM HASHTAG TAG=?',(tag,))
    PK = cur.fetchone()
    if PK is None:
        cur.execute('INSERT INTO HASHTAG VALUES (NULL, ?, NULL)',(tag,))
        conn.commit()
    
    # 무조건 태그풀에 태그가 있음
    cur.execute('SELECT PK FROM HASHTAG TAG=?',(tag,))
    PK2 = cur.fetchone()
    fkList.append(PK2)
    cur.execute('UPDATE HASHTAG SET COUNT = COUNT +1 WHERE PK=?',(PK2,))
    conn.commit()

NameError: name 'tags' is not defined

In [ ]:
# 등록 = 3 (1,2 합쳐서)
PK, fkList
for FK in fkList:
    cur.executemany('INSERT INTO POSTTAG VALUES(?,?)',(PK,FK))

# 위에 함수로 

In [ ]:
# 수정
def modifyPost(content, *tags):
    # 1. 게시글 찾는 과정 - 내용이 항상 일치한다고 가정
    cur.execute('SELECT PK FROM POST WHERE CONTENT=?', (content,))
    PPK = cur.fetchone()[0]

    # 2. 게시글에 달린 해시태그 목록
    cur.execute('SELECT TFK FROM POSTTAG WHERE PFK=?',(PPK,))
    tagList = list(cur.fetchall()) #[[1,],[]]
    for row in cur.fetchall():
        tagList.append(row[0])

    # 2. 해시태그 찾는 과정
    tpkList = list()
    for tag in tags:
        cur.execute('SELECT PK FROM HASHTAG WHERE TAG=?',(tag,))
        if cur.fetchone() is None:
            cur.execute('INSERT INTO HASHTAG VALUES (NULL, ?, NULL)',(tag,))
            conn.commit()
    
    cur.execute('SELECT PK FROM HASHTAG TAG=?',(tag,))
    TPK = cur.fetchone()[0]

    # 기존 등록된 태그와 일치하지 않으면
    if TPK not in tagList:
        cur.execute('UPDATE HASHTAG SET COUNT = COUNT +1 WHERE PK=?',(TPK,))
        cur.execute('INSERT INTO POSTTAG VALUES (?, ?)',(PPK,TPK))
        conn.commit()

    # 4. 삭제된 해시태그 찾는 과정
    for tag in tagList:
        if tag not in tpkList:
            cur.execute('UPDATE HASHTAG SET COUNT = COUNT -1 WHERE PK=?',(tag,))
            cur.execute('DELETE FROM HASHTAG WHERE PK=?',(tag,))
            conn.commit()

modifyPost('내용2','python','alchemy')